# **Lab 2: Matrix Factorization**
**Felipe Vicencio**

# **Abstract**
This report contains functions to factorize matrices, and a function for sparse-matrix multiplication.

# **About the code**

In [0]:
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Author: Felipe Vicencio Neumann
# Date: 10-2-2020

# Based on a template by Johan Hoffman:
# Copyright (C) 2019 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [0]:
from google.colab import files
import numpy as np
import random as r
import math

# **Introduction**

In this lab 3: sparce-matrix multiplication, QR factorization, and Direct $Ax = b$ solver.

# **Methods**


## Sparse-Matrix Multiplication:
Sparse Matrices are a way to store matrices using less memory than they'd normally need, by storing only the non-zero values using 3 lists:
- **val**: stores all the non-zero values in order
- **column_index**: stores the column index of each non-zero value
- **row_pointer**: starts with a 0, and then for each row is stores the number of non-zero values, plus the number of non-zero values in the rows that came before

In order to take a row from the matrix, we define **row_start** and **row_end**, which corresponds to **row_index[row_number]** and **row_index[row_number + 1]** respectively. Then, the values in the row are **val[row_start : row_end]** and their correspoindg column values are **column_indes[row_start : row_end]**.

By doing this, not only do we avoid storing the zero values, but also multiplying them, since we can use de column indexes to only take the values we need to multiply.





In [0]:
def sparse_mult(val, col_idx, row_ptr, x):
	b = np.zeros(row_ptr.size - 1)
	for row in range(row_ptr.size-1):
			row_start, row_end = row_ptr[row], row_ptr[row+1]
			row_values = val[row_start:row_end]
			columns = col_idx[row_start:row_end]
			new_value = 0
			for i in range(columns.size):
				new_value += row_values[i]*x[columns[i]]
			b[row] = new_value
	return b

## QR Factorization:
QR factorization is a method to decompose a matrix into a product of an orthogonal matrix (Q), and an upper-triangular matrix (R).

To do this, we first define the projection:

$proj_u(a )= \frac{\langle u, a\rangle}{\langle u, u\rangle}u$

then, we take columns:

$u_k = a_k - \sum_{i+1}^{k-1}proj_{u_i}(a_k)$

where $a_i$ correspond to the columns of the full column rank matrix $A = [a_1, ..., a_n]$.

Then, to get Q, we divide each $u_k$ column by its norm:

$Q = \Big( \frac{u_1}{||u_1||} \quad \frac{u_1}{||u_1||} \quad \cdots \quad \frac{u_n}{||u_n||}\Big)$

After this, it's easy to get $R$:

$Q^TQ = I\\
Q^TA = Q^TQR = R$

In [0]:
def proj(u, a):
  assert(u.ndim == a.ndim == 1)
  if abs(np.dot(u,u)) < 0.0001:
    return "error"
  frac = np.dot(u,a)/np.dot(u, u)
  return frac*u

def QR(A):
  rows, cols = A.shape
  t = np.transpose(A)   #we transpose the matrix to make it easier to take the columns
  Q = np.zeros(t.shape) #This Q corresponds to Q^t
  Q[0] = t[0] 
  for col_num in range(1, cols):
    u = t[col_num]
    for i in range(col_num):
      p =  proj(Q[i], t[col_num])
      if type(p) == str:
        return("###Error, matrix A is linearly dependent")
      else:
        u = u - p
    Q[col_num] = u
  for i in range(Q.shape[0]):
    Q[i] = Q[i]/np.linalg.norm(Q[i])
  return(np.transpose(Q), Q@A)
 

## Direct Solve:
We can use backwards substitution and QR factorization to solve $Ax = b$:

$Ax = b \\ Q^TAx = Q^Tb \\ Rx = Q^Tb$

And then we use backwards substitution with the matrix $R$ and the vector $Q^Tb$.

In [0]:
def back_sub(A, b):
		x = [b[-1]/A[-1][-1]]
		for row in reversed(range(A.shape[0]-1)):
			s = 0
			for i in range(len(x)):
				s += x[-1 - i] * A[row][-1 - i]
			x.insert(0, (b[row] - s)/A[row][row])
		return x

def direct_solve(A, b):
    Q, R = QR(A)
    Qt = np.transpose(Q)
    return back_sub(R, Qt@b)



# **Results**
The methods were tested in the following ways:

- **Sparse-matrix Multiplication**: tested against numpy multiplying the normal matrix
- **QR Factorization**: tested against Frobenius norms (square root of the sum of all elements of the matrix squared)
- **Direct Solve**: tested against np.solve()

In [0]:
# Sparse-Matrix multiplication tests:

A         = np.array([[0,0,0,0], [5,8,0,0], [0,0,3,0], [0,6,0,0]])
VAL       = np.array([5, 8, 3, 6])
COL_INDEX = np.array([0,1,2,1])
ROW_INDEX = np.array([0,0,2,3,4])
x         = np.array([2, 5, 2, 3])
assert(np.array_equal(sparse_mult(VAL, COL_INDEX, ROW_INDEX, x), A@x))

A         = np.array([[10, 20, 0, 0, 0, 0], [0, 30, 0, 40, 0, 0], [0, 0, 50, 60, 70, 0], [0, 0, 0, 0, 0, 80]])
VAL       = np.array([10, 20, 30, 40, 50, 60, 70, 80])
COL_INDEX = np.array([0, 1, 1, 3, 2, 3, 4, 5])
ROW_INDEX = np.array([0, 2, 4, 7, 8])
x         = np.array([1, 6, 3, 2, 11, 8])
assert(np.array_equal(sparse_mult(VAL, COL_INDEX, ROW_INDEX, x), A@x))

A         = np.array([[7, 7, 6, 1], [8, 8, 6, 7], [4, 2, 5, 12]])
VAL       = np.array([7, 7, 6, 1, 8, 8, 6, 7, 4, 2, 5, 12])
COL_INDEX = np.array([0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3])
ROW_INDEX = np.array([0, 4, 8, 12])
x         = np.array([3, 4, 1, 5])
assert(np.array_equal(sparse_mult(VAL, COL_INDEX, ROW_INDEX, x), A@x))

print("Sparse-Matrix multiplication test passed")


# QR Factorization tests:

A = np.array([[12, -51, 4], [6, 167, -68], [-4, 24, -41]])
Q, R = QR(A) 
assert(np.linalg.norm(Q@R - A) < 10**(-6))
assert(np.linalg.norm(np.transpose(Q)@Q - np.identity(A.shape[0]) < 10**(-6)))

A = np.array([[1, 2, 3, 4], [5, 2.3, 7, 8], [-1, -2, 0, -4], [0.2, 0, -1, -0.3]])
Q, R = QR(A) 
assert(np.linalg.norm(Q@R - A) < 10**(-6))
assert(np.linalg.norm(np.transpose(Q)@Q - np.identity(A.shape[0]) < 10**(-6)))

A = np.array([[0, 1, 0], [0, 1, 0], [0, 2, 0]])
print(QR(A)) #Should throw an error, since A is linearly dependent

print("QR Factirization test passed")


# Direct Solve tests:

A = np.array([[1, -2, 1],[0, 1, 6],[0, 0, 1]])
b = np.array([4, -1, 2])
x = direct_solve(A, b)
assert(np.linalg.norm(A@x - b) < 10**(-6))
assert(np.linalg.norm(x - np.linalg.solve(A, b)) < 10**(-6))

A = np.array([[12, -51, 4], [6, 167, -68], [-4, 24, -41]])
b = np.array([1, -1, 2])
x = direct_solve(A, b)
assert(np.linalg.norm(A@x - b) < 10**(-6))
assert(np.linalg.norm(x - np.linalg.solve(A, b)) < 10**(-6))

A = np.array([[1, 2, 3, 4], [5, 2.3, 7, 8], [-1, -2, 0, -4], [0.2, 0, -1, -0.3]])
b = np.array([3, -4, -8, 0])
x = direct_solve(A, b)
assert(np.linalg.norm(A@x - b) < 10**(-6))
assert(np.linalg.norm(x - np.linalg.solve(A, b)) < 10**(-6))

print("Direct Solve test passed")






Sparse-Matrix multiplication test passed
###Error, matrix A is linearly dependent
QR Factirization test passed
Direct Solve test passed


# Discussion

As expected, the functions give the desired results. It's worth noting that Sparse-Matrices should probably be used in the QR algorithm if we want to use it on bigger matrices, since R will always contain many zeros. By doing this, the Direct Solve algorithm could alse a little more optimized.
Some parts of this report were made in collaboration with Fabián Levican.